In [3]:
from pyspark import RDD, SparkContext
import random
from collections import defaultdict
from itertools import combinations
import sys
import math
import json
import time
from copy import deepcopy
import csv

In [4]:
train_file = "yelp_train.csv"
#train_file = "train_easy.csv"
output = "output.json"

In [5]:
test_partitions = 2
sc = SparkContext("local", "HW3").getOrCreate()
sc.setLogLevel("WARN")

In [6]:
#input_RDD = sc.textFile(train_file, test_partitions).map(lambda line: line.split(','))
input_RDD = sc.textFile(train_file).map(lambda line: line.split(','))
header = input_RDD.first()
input_RDD = input_RDD.filter(lambda data: data != header).map(lambda x: (x[0], x[1]))

Hyper Parameters

In [7]:
no_of_hash_function = 50
prime = 9999999851
hash_a = random.sample(range(1,1000000000000000000),no_of_hash_function)
hash_b = random.sample(range(1,1000000000000000000),no_of_hash_function)
b = no_of_hash_function
r = 5

# Getting User ID Indices

In [8]:
user_RDD = input_RDD.map(lambda x:x[0]).distinct().sortBy(lambda x:x).zipWithIndex()

In [9]:
biz_RDD = input_RDD.map(lambda x:x[1]).distinct().sortBy(lambda x:x).zipWithIndex()

In [10]:
rev_biz_RDD = biz_RDD.map(lambda x:(x[1],x[0])).collectAsMap()

In [11]:
user_index_map = user_RDD.collectAsMap()
biz_index_map = biz_RDD.collectAsMap()

# m is number of users
m = len(user_index_map)

In [12]:
user_index_map

{'---1lKK3aKOuomHnwAkAow': 0,
 '--2vR0DIsmQ6WfcSzKWigw': 1,
 '--BumyUHiO_7YsHurb9Hkw': 2,
 '--Qh8yKWAvIP4V4K8ZPfHA': 3,
 '--RlSfc-QmcHFGHyX6aVjA': 4,
 '--YhjyV-ce1nFLYxP49C5A': 5,
 '--ZNfWKj1VyVElRx6-g1fg': 6,
 '-0-hVEpwWEcJLJoGq3rE3g': 7,
 '-0HhZbPBlB1YZx3BhAfaEA': 8,
 '-0b84SUGVN0YkG5j2MCmBw': 9,
 '-0e6xyw_4zyg-2YtqSlS_g': 10,
 '-1wbglcr6x1qrUbqP1YAIA': 11,
 '-24Yo5J2MvjL685lPpQWjw': 12,
 '-267Yx8RmdP6io2-qI4UcQ': 13,
 '-2OB54nQ6FsGLUM-R1KXnA': 14,
 '-2UOdDlzg7XLuUuyOrldhA': 15,
 '-2gOxVWcnBr5DclrrsWXCA': 16,
 '-2kCxY7_aw5hOz7fJnGMbQ': 17,
 '-3Kq9ssSot4QizbPYO6ZQQ': 18,
 '-3PTUP443q6hQESLKSu95w': 19,
 '-3s52C4zL_DHRK0ULG6qtg': 20,
 '-4JDJeFS0YAYSiSvIshGLQ': 21,
 '-4LBQwoirPM6VPLj4Ko-Bg': 22,
 '-50XWnmQGqBgEI-9ANvLlg': 23,
 '-57uOzAWlx__p6QlX1pojA': 24,
 '-594af_E7Z9VVjQc9pJK3g': 25,
 '-5Q56Z3K0ZpaDStmOPCt5g': 26,
 '-5e4VTnu_pR4Gpv3VSncaw': 27,
 '-5ohKzWWOzAG1M1OtcoPgQ': 28,
 '-5ye1ya0wRQhNRF9NfQ1fA': 29,
 '-6559fkJ6rCWIZDbqVUomA': 30,
 '-7IMRsmV-Gt7KAK3yivZGQ': 31,
 '-7bM_DeL2Kj2CuYu

In [13]:
biz_index_map

{'--6MefnULPED_I942VcFNA': 0,
 '--7zmmkVg-IMGaXbuVd0SQ': 1,
 '--9e1ONYQuAa-CB_Rrw7Tw': 2,
 '--DaPTJW3-tB1vP-PfdTEg': 3,
 '--FBCX-N37CMYDfs790Bnw': 4,
 '--I7YYLada0tSLkORTHb5Q': 5,
 '--S62v0QgkqQaVUhFnNHrw': 6,
 '--SrzpvFLwP_YFwB_Cetow': 7,
 '--cZ6Hhc9F7VkKXxHMVZSQ': 8,
 '--cjBEbXMI2obtaRHNSFrA': 9,
 '--ujyvoQlwVoBgMYtADiLA': 10,
 '-06-Fdi30wJx-JA8P7CAng': 11,
 '-0BxAGlIk5DJAGVkpqBXxg': 12,
 '-0DET7VdEQOJVJ_v6klEug': 13,
 '-0NrB58jqKqJfuUCDupcsw': 14,
 '-0ZO00Vm2ADchytlEkoXmg': 15,
 '-0dWjxaPKrXAn8urSnkSLA': 16,
 '-0mm8pqBSIOYZQHeo8XnkA': 17,
 '-0qht1roIqleKiQkBLDkbw': 18,
 '-1dMU2kz5AhTC6N1W9xwuQ': 19,
 '-1m9o3vGRA8IBPNvNqKLmA': 20,
 '-1uVyyy3UXNA9WoPucNniA': 21,
 '-1vfRrlnNnNJ5boOVghMPA': 22,
 '-1xCh7Cocn6IwFzhELyohA': 23,
 '-1xuC540Nycht_iWFeJ-dw': 24,
 '-2QGc6Lb0R027lz0DpWN1A': 25,
 '-2ToCaDFpTNmmg3QFzxcWg': 26,
 '-2hOglg7Lh8ZgclQJ9ba2w': 27,
 '-2pQf1ceDZyE2ReCNbj-3A': 28,
 '-382X_bWhJKPJKP1VfXOmw': 29,
 '-3Yo-5oNDhJ3SFgCGfOyuw': 30,
 '-3zffZUHoY8bQjGfPSoBKQ': 31,
 '-49WY_TEa9ZEcRk_

In [14]:
"""hash_a = [1,1]
hash_b = [1,3]
no_of_hash_function = 2
primes = [1000000,100000]
b = no_of_hash_function
r = 1"""

'hash_a = [1,1]\nhash_b = [1,3]\nno_of_hash_function = 2\nprimes = [1000000,100000]\nb = no_of_hash_function\nr = 1'

In [15]:
def minHash(review_row):
    hash_code = []
    for i in range(b):
        hash_code.append(((hash_a[i]*review_row + hash_b[i])%prime) % (m))

    return hash_code

In [16]:
hashed_indices = user_RDD.map(lambda x: (user_index_map[x[0]], minHash(x[1])))

In [17]:
def map_to_indices(original, mapped_indices):
    indexes = []
    for i in original:
        indexes.append(mapped_indices[i])
    return indexes

In [18]:
user_biz_indices = input_RDD.groupByKey().map(lambda x: (x[0], set(x[1])))\
        .map(lambda x: (user_index_map[x[0]], [biz_index_map[i] for i in x[1]]))

In [19]:
biz_user_indices = input_RDD.map(lambda x: (x[1],x[0])).groupByKey().map(lambda x: (x[0], set(x[1])))\
        .map(lambda x: (biz_index_map[x[0]], [user_index_map[i] for i in x[1]]))

In [20]:
signatures_users = user_biz_indices.join(hashed_indices)
signatures_users = signatures_users.map(lambda x: (x[1][1], x[1][0])).flatMapValues(lambda x:x)

In [21]:
single_signatures = signatures_users.map(lambda x: (x[1], x[0])).reduceByKey(lambda x, y: list(map(min, zip(x, y))))

In [22]:
def band_split(input):
    split = []
    for num, row in enumerate(range(0, b, r)):
        split.append((num, tuple(input[row:row + r])))
    return split

In [23]:
def split_buckets(buckets):
    output = []

    bin_num = buckets[0]
    for pairs in buckets[1]:
        combined_pair = (pairs[0],pairs[1][0])
        output.append((combined_pair,bin_num))
    return(output)

In [24]:
candidate_singletons = single_signatures.map(lambda x: (x[0], band_split(x[1]))).map(lambda x:split_buckets(x)).flatMap(lambda x:x).groupByKey().map(lambda x:list(x[1])).filter(lambda x: len(x)>1)

In [25]:
candidate_pairs = candidate_singletons.flatMap(lambda x: combinations(x,2)).distinct()

In [26]:
biz_user_indices = biz_user_indices.collectAsMap()

In [27]:
def calc_jaccard_similarity(pair):

    value = len(set(biz_user_indices[pair[0]])&set(biz_user_indices[pair[1]])) / len(set(biz_user_indices[pair[0]])|set(biz_user_indices[pair[1]]))
    return value

In [79]:
from pyspark import RDD, SparkContext, SparkConf
import random
from collections import defaultdict
from itertools import combinations
import sys
import math
import json
import time
from copy import deepcopy
import csv


start_time = time.time()
train_file = "yelp_train.csv"
#train_file = "train_easy.csv"
output = "out.csv"
partitions = 1


#sc = SparkContext("local", "HW3").getOrCreate()

sc.setLogLevel("Error")

input_RDD = sc.textFile(train_file, partitions).map(lambda line: line.split(','))
header = input_RDD.first()
input_RDD = input_RDD.filter(lambda data: data != header).map(lambda x: (x[0], x[1]))

no_of_hash_function = 36
prime = 9999999851
b = no_of_hash_function
r = 4

#hash_a = random.sample(range(1,100000000000),no_of_hash_function)
hash_a = [37252355246, 26619390995, 89720066986, 58729152086, 65115724005, 14257466205, 35785832864, 28129429934, 17116188388, 27338336169, 7511209268, 93070171028, 69142787049, 89444985967, 7065188552, 79295033333, 97325688096, 46792013639, 25942372456, 52008711403, 23223739655, 58749885760, 99435808567, 48246658288, 84037348945, 65984214450, 14852191996, 52257930830, 66037436407, 17079807874, 35283692389, 59489573578, 85071625896, 95792007828, 70137659362, 61850771791]
#hash_b = random.sample(range(1,100000000000),no_of_hash_function)
hash_b =  [3667522832, 80932870561, 78162357018, 26991468314, 25065354953, 13471083481, 62649240755, 38610145150, 722779167, 12888869643, 20389440915, 96581956888, 64897392981, 75860525074, 84426924707, 61000963557, 14107400107, 50819347909, 17419209729, 73110090776, 44522547106, 49779693221, 1558892803, 52567622259, 2293493494, 15045222107, 60732212600, 23355110027, 36640867841, 25346067470, 69818330326, 75122082653, 73769383823, 15573922425, 92618641131, 84312445529]


jsonList = []

jsonList.append(hash_a)
jsonList.append(hash_b)
with open('data.json', 'w') as f:
    json.dump(jsonList, f)



user_RDD = input_RDD.map(lambda x:x[0]).distinct().sortBy(lambda x:x).zipWithIndex()
biz_RDD = input_RDD.map(lambda x:x[1]).distinct().sortBy(lambda x:x).zipWithIndex()

rev_biz_RDD = biz_RDD.map(lambda x:(x[1],x[0])).collectAsMap()

user_index_map = user_RDD.collectAsMap()
biz_index_map = biz_RDD.collectAsMap()


m = len(user_index_map)


def minHash(review_row):
    hash_code = []
    for i in range(b):
        hash_code.append(((hash_a[i]*review_row + hash_b[i])%prime) % (m))

    return hash_code


hashed_indices = user_RDD.map(lambda x: (user_index_map[x[0]], minHash(x[1])))


def map_to_indices(original, mapped_indices):
    indexes = []
    for i in original:
        indexes.append(mapped_indices[i])
    return indexes


user_biz_indices = input_RDD.groupByKey().map(lambda x: (x[0], set(x[1]))).map(lambda x: (user_index_map[x[0]], [biz_index_map[i] for i in x[1]]))

biz_user_indices = input_RDD.map(lambda x: (x[1],x[0])).groupByKey().map(lambda x: (x[0], set(x[1]))).map(lambda x: (biz_index_map[x[0]], [user_index_map[i] for i in x[1]]))

signatures_users = user_biz_indices.join(hashed_indices)
signatures_users = signatures_users.map(lambda x: (x[1][1], x[1][0])).flatMapValues(lambda x:x)

single_signatures = signatures_users.map(lambda x: (x[1], x[0])).reduceByKey(lambda x, y: list(map(min, zip(x, y))))


def band_split(input):
    split = []
    for num, row in enumerate(range(0, b, r)):
        split.append((num, tuple(input[row:row + r])))
    return split


def split_buckets(buckets):
    output = []

    bin_num = buckets[0]
    for pairs in buckets[1]:
        combined_pair = (pairs[0],pairs[1][0])
        output.append((combined_pair,bin_num))
    return output


candidate_singletons = single_signatures.map(lambda x: (x[0], band_split(x[1]))).map(lambda x:split_buckets(x)).flatMap(lambda x:x).groupByKey().map(lambda x:list(x[1])).filter(lambda x: len(x)>1)
candidate_pairs = candidate_singletons.flatMap(lambda x: combinations(x,2)).distinct().sortBy(lambda x:x)

biz_user_indices = biz_user_indices.collectAsMap()


def calc_jaccard_similarity(pair):

    value = len(set(biz_user_indices[pair[0]])&set(biz_user_indices[pair[1]])) / len(set(biz_user_indices[pair[0]])|set(biz_user_indices[pair[1]]))
    return value


def review_partitioner(key):
    return ord(key[:1])


x = candidate_pairs.partitionBy(20, review_partitioner)
x.getNumPartitions()
similar_biz = candidate_pairs.map(lambda x: ((rev_biz_RDD[x[0]],rev_biz_RDD[x[1]]),calc_jaccard_similarity(x))).filter(lambda x:x[1]>=0.5).collect()

with open(output, 'w+') as f:
    f.write('business_id_1, business_id_2, similarity\n')

    for i in similar_biz:
        f.write(str(i[0][0]) + "," + str(i[0][1]) + "," + str(i[1]) + "\n")



print("Time Taken = ", time.time() - start_time )

Time Taken =  111.00083303451538


In [92]:
x

[[['-8O4kt8AIRhM3OUxt-pWLg', '_p64KqqRmPwGKhZ-xZwhtg'], 0.5],
 [['-A5jntJgFglQ6zwAmOiOMw', 'cTqIuG-fvlQQL0OWzsFdig'], 0.5],
 [['-Jhlh8Scjy669NdtCfKSSg', 'o5Mofj5KJkYAMs_fhxftpg'], 0.5],
 [['-LfTBo0oa_uD454ScEW2XA', 'J5U-nbhKSnnX7DJGT6QELg'], 1.0],
 [['-LfTBo0oa_uD454ScEW2XA', 'Lwb6bG1Qu3BbW7FJj5suLw'], 0.5],
 [['-LfTBo0oa_uD454ScEW2XA', 'aokqWHt8vMf5iwj8KZZ5eg'], 0.5],
 [['-Mm02AeY1PMGg-l-ShMxUg', 'RWwvuTbnf3DleUSiFbUscg'], 0.5],
 [['-Mm02AeY1PMGg-l-ShMxUg', 'cKgkSMcPXwWTzPrJRpa2qw'], 0.5],
 [['-N_YCDH4HijYnJ-RisQfHA', 'W2Vis19kUa7kP6GkSvJ9zQ'], 0.5],
 [['-N_YCDH4HijYnJ-RisQfHA', 'zm8u9z2KCwEkNiCD59UQ3Q'], 0.5],
 [['-UV2tt3gy57-5a29paA_4w', 'HUMdEqtlPF70AqyZmiyAhA'], 0.5],
 [['-ePLgQ_afOTW1STxD-2RIA', 'fBU5QssrXMXPbJWD08o9zg'], 0.5],
 [['-fn65iT4z-nh1Ybfyajxng', '9BQGpsIRhs9cRHuGh9xuJw'], 0.5],
 [['-i3dOjumvOw-52aGXU1xDg', '6nMYROXu0VX4Ytpdsfi3XA'], 0.5],
 [['-ml57G52t4mQpYRHe54HWQ', 'Nm9xRoLxn4a_q4XlVAJfSg'], 0.5],
 [['06X34OSshQJPDiyUDHtS0A', 'or4yipilbCkKIFnF1RP5Ow'], 0.5],
 [['06X3

In [96]:
s = time.time()
ordered = []
for i in similar_biz:
    #print(i)
    ordered.append(list((sorted(i[0]),i[1])))

x = sorted(ordered, key= lambda x: (x[0]))
print(time.time()-s)

0.001725912094116211


In [80]:
similar_biz

[(('-A5jntJgFglQ6zwAmOiOMw', 'cTqIuG-fvlQQL0OWzsFdig'), 0.5),
 (('-Jhlh8Scjy669NdtCfKSSg', 'o5Mofj5KJkYAMs_fhxftpg'), 0.5),
 (('-LfTBo0oa_uD454ScEW2XA', 'J5U-nbhKSnnX7DJGT6QELg'), 1.0),
 (('-LfTBo0oa_uD454ScEW2XA', 'aokqWHt8vMf5iwj8KZZ5eg'), 0.5),
 (('-Mm02AeY1PMGg-l-ShMxUg', 'cKgkSMcPXwWTzPrJRpa2qw'), 0.5),
 (('-ePLgQ_afOTW1STxD-2RIA', 'fBU5QssrXMXPbJWD08o9zg'), 0.5),
 (('-fn65iT4z-nh1Ybfyajxng', '9BQGpsIRhs9cRHuGh9xuJw'), 0.5),
 (('-ml57G52t4mQpYRHe54HWQ', 'Nm9xRoLxn4a_q4XlVAJfSg'), 0.5),
 (('0JoteSDMdGJSgYzk60vSdQ', 'NjypbHnZG6HjNQVDVQ_iYA'), 0.5),
 (('0OOmvIS1Tb5ccYOCvq4Yeg', 'lnU-G15oaYWwqqbUXj9JEA'), 0.5),
 (('0TUqA5zzBpj8izS96M_50A', 'PMTrNqHunjLgUfnuALUnmQ'), 0.5),
 (('0l_HQpZ4gsR5T6Ejqcgi2Q', 'DTmYVvujYjELUgLfPRtN7g'), 0.5),
 (('0zC1EGTJFd5_xPjHruQMZQ', 'TfgGLuX0JBr-KKXatxQWFQ'), 0.5),
 (('12g_z3nFKbN2telbzG5ZWQ', '2FQ4fhO6_gXCYRDsw5wtQw'), 0.5),
 (('157Y-QIXWnqQra78_WfW5g', 'e3rO7PfdGrdHPzayPtoi_A'), 1.0),
 (('1AyaRLA83q_V9D3bYevRkQ', '8m1HM54_hE6UcQcxWTWMaA'), 0.5),
 (('1FGe

In [74]:
sc.broadcast(biz_user_indices)

In [70]:
def calc_jaccard_similarity(pair):

    value = len(set(biz_user_indices[pair[0]])&set(biz_user_indices[pair[1]])) / len(set(biz_user_indices[pair[0]])|set(biz_user_indices[pair[1]]))

    yield value

In [78]:
candidate_pairs.collect()

[(0, 105),
 (0, 218),
 (0, 470),
 (0, 533),
 (0, 540),
 (0, 589),
 (0, 981),
 (0, 1137),
 (0, 1733),
 (0, 1880),
 (0, 1953),
 (0, 2208),
 (0, 2227),
 (0, 2437),
 (0, 2764),
 (0, 2853),
 (0, 3021),
 (0, 3201),
 (0, 3229),
 (0, 3277),
 (0, 3379),
 (0, 3425),
 (0, 3653),
 (0, 3672),
 (0, 3875),
 (0, 3895),
 (0, 3951),
 (0, 4001),
 (0, 4043),
 (0, 4073),
 (0, 4083),
 (0, 4233),
 (0, 4727),
 (0, 5585),
 (0, 5865),
 (0, 6079),
 (0, 6201),
 (0, 6339),
 (0, 6551),
 (0, 7241),
 (0, 7251),
 (0, 7308),
 (0, 7406),
 (0, 7606),
 (0, 7638),
 (0, 7789),
 (0, 7815),
 (0, 7909),
 (0, 7913),
 (0, 8499),
 (0, 8560),
 (0, 8687),
 (0, 8859),
 (0, 8877),
 (0, 9003),
 (0, 9045),
 (0, 9153),
 (0, 9567),
 (0, 9656),
 (0, 9681),
 (0, 9917),
 (0, 10123),
 (0, 10225),
 (0, 10429),
 (0, 10483),
 (0, 10565),
 (0, 10779),
 (0, 11155),
 (0, 11488),
 (0, 11531),
 (0, 11599),
 (0, 12139),
 (0, 12527),
 (0, 12615),
 (0, 12653),
 (0, 12949),
 (0, 12979),
 (0, 12991),
 (0, 12997),
 (0, 13333),
 (0, 13445),
 (0, 13787),
 (

In [76]:
similar_biz = candidate_pairs.mapPartitions(lambda x:calc_jaccard_similarity(x)).mapValues(lambda x:dict(x))

In [56]:
x = candidate_pairs.partitionBy(20, review_partitioner)

In [57]:
x.getNumPartitions()

20

In [58]:
similar_biz = candidate_pairs.map(lambda x: (sorted((rev_biz_RDD[x[0]],rev_biz_RDD[x[1]])),calc_jaccard_similarity(x))).filter(lambda x:x[1]>=0.5).sortBy(lambda x: x).collect()

In [42]:
len(similar_biz)

644

In [43]:
similar_biz

[(['-8O4kt8AIRhM3OUxt-pWLg', '_p64KqqRmPwGKhZ-xZwhtg'], 0.5),
 (['-A5jntJgFglQ6zwAmOiOMw', 'cTqIuG-fvlQQL0OWzsFdig'], 0.5),
 (['-Jhlh8Scjy669NdtCfKSSg', 'o5Mofj5KJkYAMs_fhxftpg'], 0.5),
 (['-LfTBo0oa_uD454ScEW2XA', 'J5U-nbhKSnnX7DJGT6QELg'], 1.0),
 (['-LfTBo0oa_uD454ScEW2XA', 'Lwb6bG1Qu3BbW7FJj5suLw'], 0.5),
 (['-LfTBo0oa_uD454ScEW2XA', 'aokqWHt8vMf5iwj8KZZ5eg'], 0.5),
 (['-Mm02AeY1PMGg-l-ShMxUg', 'RWwvuTbnf3DleUSiFbUscg'], 0.5),
 (['-Mm02AeY1PMGg-l-ShMxUg', 'cKgkSMcPXwWTzPrJRpa2qw'], 0.5),
 (['-N_YCDH4HijYnJ-RisQfHA', 'W2Vis19kUa7kP6GkSvJ9zQ'], 0.5),
 (['-N_YCDH4HijYnJ-RisQfHA', 'zm8u9z2KCwEkNiCD59UQ3Q'], 0.5),
 (['-UV2tt3gy57-5a29paA_4w', 'HUMdEqtlPF70AqyZmiyAhA'], 0.5),
 (['-ePLgQ_afOTW1STxD-2RIA', 'fBU5QssrXMXPbJWD08o9zg'], 0.5),
 (['-fn65iT4z-nh1Ybfyajxng', '9BQGpsIRhs9cRHuGh9xuJw'], 0.5),
 (['-i3dOjumvOw-52aGXU1xDg', '6nMYROXu0VX4Ytpdsfi3XA'], 0.5),
 (['-ml57G52t4mQpYRHe54HWQ', 'Nm9xRoLxn4a_q4XlVAJfSg'], 0.5),
 (['06X34OSshQJPDiyUDHtS0A', 'or4yipilbCkKIFnF1RP5Ow'], 0.5),
 (['06X3

In [44]:
with open(output, 'w+') as f:
    f.write('business_id_1, business_id_2, similarity\n')

    for i in similar_biz:
        f.write(str(i[0][0]) + "," + str(i[0][1]) + "," + str(i[1]) + "\n")



Takes long but 644

In [ ]:
[[898405288, 933808915, 403825155, 240906728, 615172511, 191582422, 298817609, 322316527, 395742027, 105777651, 599450907, 381921761, 807204451, 250141610, 334128369, 529253420, 749284893, 558504459, 785397853, 618297487, 968633756, 768087632, 226827357, 618755213, 35542039, 485931754, 994705780, 977292401, 128968581, 100188395, 600722250, 106697013, 38006137, 761445677, 465265710, 727728418, 711948351, 445445396, 683459037, 257205534, 862024064, 193079182], [362385852, 639717109, 160239503, 592124262, 33379744, 461192476, 240054888, 858723043, 777214979, 399988454, 648576487, 673280016, 634962968, 827103018, 766973178, 537098107, 946986407, 623739365, 182058237, 24151048, 670320886, 881135274, 358273162, 35449653, 979951333, 706106929, 609477233, 508609989, 647151776, 629840460, 816781456, 289999321, 386712543, 628552086, 757053945, 44383110, 53586658, 291590854, 5283593, 226859799, 557097954, 360961267]]

32,4, 87s, 644

In [ ]:
[[561411597, 536102609, 962041843, 88769901, 402263562, 576522546, 646255122, 426196570, 986279315, 730930538, 938238327, 46657990, 625668113, 776142372, 365910492, 220025087, 438779309, 257509626, 51008110, 858953963, 482941730, 188348701, 163316105, 550492038, 625893709, 55474162, 626357495, 192688749, 507595192, 296511066, 795655686, 237087453], [681544245, 367734889, 18301630, 640272823, 514026240, 504676207, 755097963, 28334507, 106029267, 970666194, 32839564, 546889339, 507441094, 652043990, 750651881, 841057864, 554919253, 466206071, 380505048, 165484463, 991883825, 929473186, 115225557, 690490609, 988761142, 709070008, 36925323, 542805195, 284957411, 374263359, 464689893, 261102111]]

In [ ]:
645 40 5
[[456037669, 192322267, 687260855, 691710074, 284304235, 773401520, 578577226, 16076371, 169754507, 274078022, 717601821, 458376655, 866372531, 611485188, 898371082, 377708098, 355748898, 909874651, 343534834, 524585081, 840166716, 31782054, 642604087, 595101646, 474158591, 677298515, 229344992, 881399585, 618697588, 591738175, 579518918, 773056874, 127767902, 297545991, 394654804, 668018045, 251599084, 988775148, 803888594, 320010317], [886156019, 209670011, 875688612, 515818644, 38471359, 528515980, 653179753, 52787863, 788206968, 876878148, 185216275, 701387621, 124677045, 195467667, 408683626, 465707914, 226740969, 62793358, 219481200, 830505962, 736291937, 657267618, 374771205, 811322938, 937407845, 560763659, 20617781, 191073493, 559409870, 99610144, 870596552, 679799308, 317920724, 386582966, 556039861, 476322298, 6024929, 464816763, 781831307, 304501949]]

# FIANL

In [ ]:
36 4 645
[[37252355246, 26619390995, 89720066986, 58729152086, 65115724005, 14257466205, 35785832864, 28129429934, 17116188388, 27338336169, 7511209268, 93070171028, 69142787049, 89444985967, 7065188552, 79295033333, 97325688096, 46792013639, 25942372456, 52008711403, 23223739655, 58749885760, 99435808567, 48246658288, 84037348945, 65984214450, 14852191996, 52257930830, 66037436407, 17079807874, 35283692389, 59489573578, 85071625896, 95792007828, 70137659362, 61850771791],

 [3667522832, 80932870561, 78162357018, 26991468314, 25065354953, 13471083481, 62649240755, 38610145150, 722779167, 12888869643, 20389440915, 96581956888, 64897392981, 75860525074, 84426924707, 61000963557, 14107400107, 50819347909, 17419209729, 73110090776, 44522547106, 49779693221, 1558892803, 52567622259, 2293493494, 15045222107, 60732212600, 23355110027, 36640867841, 25346067470, 69818330326, 75122082653, 73769383823, 15573922425, 92618641131, 84312445529]]